In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

In [ ]:
optimizer = torch.optim.AdamW(self.parameters(), lr=lr_config['init'],
                                    weight_decay=self.hparams.weight_decay, amsgrad=True)

scheduler = LambdaLR(optimizer,
                lr_lambda=functools.partial(
                    lambda_lr_wrapper, lr_config=lr_config, batch_size=self.hparams.batch_size))

exp.global_var_manager.register_variable('skip_backward', False)


# Do gradient clipping in the training steps
grad_clip_val = self.hparams.get('grad_clip', 1000.)

if grad_clip_val == "inspect":
    from pytorch_lightning.utilities.grads import grad_norm
    grad_dict = grad_norm(self, 'inf')      # Get the maximum absolute value.
    print(grad_dict)
    grad_clip_val = 1000.

# torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=grad_clip_val)
torch.nn.utils.clip_grad_value_(self.parameters(), clip_value=grad_clip_val)

# If detect nan values, then this step is skipped
has_nan_value_cnt = 0
for p in filter(lambda p: p.grad is not None, self.parameters()):
    if torch.any(p.grad.data != p.grad.data):
        has_nan_value_cnt += 1
if has_nan_value_cnt > 0:
    exp.logger.warning(f"{has_nan_value_cnt} parameters get nan-gradient -- this step will be skipped.")
    for p in filter(lambda p: p.grad is not None, self.parameters()):
        p.grad.data.zero_()


    def train_dataloader(self):
        # Note:
        import xcube.data as dataset
        train_set = dataset.build_dataset(
            self.hparams.train_dataset, self.get_dataset_spec(), self.hparams, self.hparams.train_kwargs)
        torch.manual_seed(0)
        return DataLoader(train_set, batch_size=self.hparams.batch_size // self.trainer.world_size, shuffle=True,
                          num_workers=self.hparams.train_val_num_workers, collate_fn=self.get_collate_fn())

    def val_dataloader(self):
        import xcube.data as dataset
        val_set = dataset.build_dataset(
            self.hparams.val_dataset, self.get_dataset_spec(), self.hparams, self.hparams.val_kwargs)
        return DataLoader(val_set, batch_size=self.hparams.batch_size // self.trainer.world_size, shuffle=False,
                          num_workers=self.hparams.train_val_num_workers, collate_fn=self.get_collate_fn())

    def test_dataloader(self):
        import xcube.data as dataset
        self.hparams.test_kwargs.resolution = self.hparams.resolution # ! use for testing when training on X^3 but testing on Y^3

        test_set = dataset.build_dataset(
            self.hparams.test_dataset, self.get_dataset_spec(), self.hparams, self.hparams.test_kwargs)
        if self.hparams.test_set_shuffle:
            torch.manual_seed(0)
        return DataLoader(test_set, batch_size=1, shuffle=self.hparams.test_set_shuffle, 
                          num_workers=0, collate_fn=self.get_collate_fn())
